# Local Model Serving with MLflow

In [1]:
import httpx 
import json
import mlflow
import pandas as pd 
import numpy as np
from examples.digit_recognition.data import get_train_test_data
from examples.digit_recognition.data import transform_to_image

from mlflow_for_ml_dev.src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(uri=(get_project_root() / 'mlruns').as_uri())

# Scoring Digit recognizer

```shell
mlflow models serve --options
```

To run the code below make sure you deploy the model using.

`poetry run mlflow models serve --model-uri models:/Digit_Recognition_Model@production --env-manager local`

In [7]:
_, x_test, _, y_test = get_train_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]

payload = {
    "instances": {"image_input": samples.tolist()},
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

if response.status_code == 200:
    predictions = response.json().get("predictions")
    pred = np.argmax(predictions, axis=-1)
    print(pd.DataFrame({"predictions": pred, "y_test": y_test[0:n_samples]}))
    
else:
    raise Exception(f"Error: {response.status_code} - {response.text}")

      predictions  y_test
5457            8       8


## Signature Validation

In [15]:
_, x_test, _, y_test = get_train_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]

payload = {
    "instances": {"image_inputs": samples.tolist()}, # invalid input name
    # "instances": {"image_input": samples.tolist()}, # valid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [22]:
print(response.json()["message"])

Failed to predict data '{'image_input': array([[[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0

## Changing the input shape

In [ ]:
_, x_test, _, y_test = get_train_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]
samples = samples.reshape(28, 28, 1) # invalid input shape
payload = {
    "instances": {"image_input": samples.tolist()}, # invalid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [21]:
print(response.json()["message"])

Failed to predict data '{'image_input': array([[[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0

## Using Inputs Field as Key for the payload

In [28]:
_, x_test, _, y_test = get_train_test_data()
x_test = transform_to_image(x_test)

url = "http://127.0.0.1:5000/invocations"
n_samples = 1
samples = x_test[0:n_samples]
payload = {
    "inputs": {"image_input": samples.tolist()}, # invalid input name
}

headers = {"Content-Type": "application/json"}
response = httpx.post(url, data=json.dumps(payload), headers=headers)

In [27]:
response.json()

{'predictions': [[0.005412942264229059,
   0.026839114725589752,
   0.04627617076039314,
   0.0038798940367996693,
   0.011171405203640461,
   0.009528354741632938,
   0.0035955572966486216,
   0.0017295869765803218,
   0.8885131478309631,
   0.0030538735445588827]]}